# Multivariable Linear Regression with Dummy Variables 


In [19]:
import pandas as pd
import numpy as np
%matplotlib inline

In [20]:

df = pd.read_csv("diabetes.csv")
df = df.dropna()
df.head()

,id,chol,stab.glu,hdl,ratio,glyhb,location,age,gender,height,weight,frame,bp.1s,bp.1d,waist,hip
0,1000,203.0,82,56.0,3.6,4.31,Buckingham,46,female,62.0,121.0,medium,118.0,59.0,29.0,38.0
1,1001,165.0,97,24.0,6.9,4.44,Buckingham,29,female,64.0,218.0,large,112.0,68.0,46.0,48.0
2,1002,228.0,92,37.0,6.2,4.64,Buckingham,58,female,61.0,256.0,large,190.0,92.0,49.0,57.0
3,1003,78.0,93,12.0,6.5,4.63,Buckingham,67,male,67.0,119.0,large,110.0,50.0,33.0,38.0
4,1005,249.0,90,28.0,8.9,7.72,Buckingham,64,male,68.0,183.0,medium,138.0,80.0,44.0,41.0


In [21]:
# Let's examine if any of these columns are highly correlated
df.corr()

,id,chol,stab.glu,hdl,ratio,glyhb,age,height,weight,bp.1s,bp.1d,waist,hip
id,1.000000,0.072507,0.022112,0.033753,-0.008901,0.019733,0.000349,-0.015348,-0.018147,0.041172,0.070621,-0.025102,0.029865
chol,0.072507,1.000000,0.164405,0.171236,0.483565,0.269842,0.240871,-0.062792,0.079947,0.200650,0.160299,0.144037,0.097619
stab.glu,0.022112,0.164405,1.000000,-0.180344,0.298967,0.741136,0.278911,0.082073,0.188542,0.152249,0.024285,0.233580,0.145520
hdl,0.033753,0.171236,-0.180344,1.000000,-0.690250,-0.169825,0.000007,-0.068437,-0.282887,0.029099,0.072706,-0.278275,-0.222435
ratio,-0.008901,0.483565,0.298967,-0.690250,1.000000,0.354605,0.171665,0.070801,0.278842,0.105523,0.034461,0.315485,0.208002
glyhb,0.019733,0.269842,0.741136,-0.169825,0.354605,1.000000,0.332658,0.051625,0.167317,0.195646,0.045604,0.247412,0.152723
age,0.000349,0.240871,0.278911,0.000007,0.171665,0.332658,1.000000,-0.097345,-0.046319,0.433331,0.057954,0.170221,0.018802
height,-0.015348,-0.062792,0.082073,-0.068437,0.070801,0.051625,-0.097345,1.000000,0.243351,-0.044820,0.044014,0.041812,-0.117486
weight,-0.018147,0.079947,0.188542,-0.282887,0.278842,0.167317,-0.046319,0.243351,1.000000,0.095969,0.180594,0.851909,0.829320
bp.1s,0.041172,0.200650,0.152249,0.029099,0.105523,0.195646,0.433331,-0.044820,0.095969,1.000000,0.617100,0.209619,0.152247


In [22]:
# Since weight, waist, and hip are all colinear, 
# let's get reduce the number of dimensions for our model

df = df[['id', 'chol', 'stab.glu', 'hdl', 'ratio', 'glyhb', 'location', 'age',
       'gender', 'height', 'weight', 'frame', 'bp.1s', 'bp.1d']]

df.head()

,id,chol,stab.glu,hdl,ratio,glyhb,location,age,gender,height,weight,frame,bp.1s,bp.1d
0,1000,203.0,82,56.0,3.6,4.31,Buckingham,46,female,62.0,121.0,medium,118.0,59.0
1,1001,165.0,97,24.0,6.9,4.44,Buckingham,29,female,64.0,218.0,large,112.0,68.0
2,1002,228.0,92,37.0,6.2,4.64,Buckingham,58,female,61.0,256.0,large,190.0,92.0
3,1003,78.0,93,12.0,6.5,4.63,Buckingham,67,male,67.0,119.0,large,110.0,50.0
4,1005,249.0,90,28.0,8.9,7.72,Buckingham,64,male,68.0,183.0,medium,138.0,80.0


In [23]:
# As you can see, 'location', 'gender', and 'frame' are categorical (or discrete) variables
# Linear regression can only work with numeric values, so we need to convert our categorical variables
# into numeric

# Let's start with gender
# Because gender only has two values, we can assign male = 0, female = 1
# We'll call this new column gender_dummy
# In terms of how we interpret this data, we can say that 1 IS female and 0 IS NOT female
df["gender_dummy"] = df.gender.map({'male':0, 'female':1})
df.head()

,id,chol,stab.glu,hdl,ratio,glyhb,location,age,gender,height,weight,frame,bp.1s,bp.1d,gender_dummy
0,1000,203.0,82,56.0,3.6,4.31,Buckingham,46,female,62.0,121.0,medium,118.0,59.0,1
1,1001,165.0,97,24.0,6.9,4.44,Buckingham,29,female,64.0,218.0,large,112.0,68.0,1
2,1002,228.0,92,37.0,6.2,4.64,Buckingham,58,female,61.0,256.0,large,190.0,92.0,1
3,1003,78.0,93,12.0,6.5,4.63,Buckingham,67,male,67.0,119.0,large,110.0,50.0,0
4,1005,249.0,90,28.0,8.9,7.72,Buckingham,64,male,68.0,183.0,medium,138.0,80.0,0


In [24]:
df["frame"].value_counts()

medium    173
small      98
large      96
Name: frame, dtype: int64

In [25]:
# Now let's take a look at frame.
# Let's see how many categories the 'frame' column contains:
df.groupby(['frame']).agg({'frame':'count'})

,frame
frame,
large,96
medium,173
small,98


## Ordered vs. Unordered Categories

As you can see from the output, 'frame' contains the following categories: large, medium, small

We cannot code it as 0=small, 1=medium, 2=large because that would imply an ordered relationship between medium and large.  If we did that, large would somehow be "twice" the medium category

**Ordered categories**
* i.e., strongly disagree, disagree, neutral, agree, strongly agree
* Can use a single dummy variable and represent the categories numerically (such as 1, 2, 3, 4, 5)

Our 'frame' feature is unordered, so we have to create additional dummy variables. We can do this using pandas:

In [26]:
# create three dummy variables using get_dummies
frame_dummies = pd.get_dummies(df["frame"], prefix='frame')
frame_dummies.head()

,frame_large,frame_medium,frame_small
0,0,1,0
1,1,0,0
2,1,0,0
3,1,0,0
4,0,1,0


In [27]:
# concatenate the dummy variable columns onto the DataFrame (axis=0 means rows, axis=1 means columns)
df = pd.concat([df, frame_dummies], axis=1)
df.head()

,id,chol,stab.glu,hdl,ratio,glyhb,location,age,gender,height,weight,frame,bp.1s,bp.1d,gender_dummy,frame_large,frame_medium,frame_small
0,1000,203.0,82,56.0,3.6,4.31,Buckingham,46,female,62.0,121.0,medium,118.0,59.0,1,0,1,0
1,1001,165.0,97,24.0,6.9,4.44,Buckingham,29,female,64.0,218.0,large,112.0,68.0,1,1,0,0
2,1002,228.0,92,37.0,6.2,4.64,Buckingham,58,female,61.0,256.0,large,190.0,92.0,1,1,0,0
3,1003,78.0,93,12.0,6.5,4.63,Buckingham,67,male,67.0,119.0,large,110.0,50.0,0,1,0,0
4,1005,249.0,90,28.0,8.9,7.72,Buckingham,64,male,68.0,183.0,medium,138.0,80.0,0,0,1,0


In [28]:
df.keys()

df.iloc[[2]]

,id,chol,stab.glu,hdl,ratio,glyhb,location,age,gender,height,weight,frame,bp.1s,bp.1d,gender_dummy,frame_large,frame_medium,frame_small
2,1002,228.0,92,37.0,6.2,4.64,Buckingham,58,female,61.0,256.0,large,190.0,92.0,1,1,0,0


In [29]:
# Unlike with simple linear regression where we use one predictor variable to predict one response variable,
# with multivariable regression we can use multiple predictor variables to predict one response variable.

X = df[['stab.glu', 'hdl', 'ratio', 'glyhb', 'age',
       'height', 'weight', 'bp.1s', 'bp.1d', 'gender_dummy',
       'frame_large', 'frame_medium', 'frame_small']]
y = df[['chol']]

In [38]:
# Split Data
# Now we can split our data into a training and test set.  In this example, we are using an 80/20 split, 
# where 80% of our data will be used for training our model, and 20% of our data will be used for testing.
    
from sklearn.model_selection import train_test_split

# Split X and y into X_
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=1)

In [39]:
# Train Model
# Now we train our LinearRegression model using the training subset of data.

from sklearn.linear_model import LinearRegression

lm = LinearRegression()
lm.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [40]:
# Now that our model is trained, we can view the coefficients of the model using regression_model.coef_, 
# which is an array of tuples of coefficients.
# Each regression coefficient shows the strength of the relationship between the predictor variable and the
# outcome variable while controlling for the other predictor variable 

for idx, col_name in enumerate(X_train.columns):
    print("The coefficient for {} is {}".format(col_name, lm.coef_[0][idx]))

The coefficient for stab.glu is -0.05960636352834215
The coefficient for hdl is 2.313942342612747
The coefficient for ratio is 27.648128714299666
The coefficient for glyhb is 1.1382567665236802
The coefficient for age is 0.10588021233754574
The coefficient for height is -1.0546153948852885
The coefficient for weight is 0.025988773709302304
The coefficient for bp.1s is 0.033029708752970954
The coefficient for bp.1d is 0.11169695449322603
The coefficient for gender_dummy is 0.8292960809564525
The coefficient for frame_large is -0.6724566160902197
The coefficient for frame_medium is 2.972500870891305
The coefficient for frame_small is -2.3000442548010858


In [41]:
# regression_model.intercept_ returns an array of intercepts
intercept = lm.intercept_[0]

print("The intercept for our model is {}".format(intercept))

The intercept for our model is 10.604682860749051


In [42]:
# R-squared  can be determined using our test set and the model’s score method.

lm.score(X_test, y_test)



0.6594346499852195

In [43]:
# We can get the mean squared error using scikit-learn’s mean_squared_error method 
# and comparing the prediction for the test data set (data not used for training) 
# with the ground truth for the data test set.

# We'll start with calculating the Mean Squared Error (MSE)

from sklearn.metrics import mean_squared_error

y_predict = lm.predict(X_test)

regression_model_mse = mean_squared_error(y_predict, y_test)

regression_model_mse

635.28970834762276

In [44]:
# And now we can calculate the Root Mean Squared Error (RMSE)
import math

math.sqrt(regression_model_mse)

25.204954043751453

In [47]:
# Now, let's try to make a prediction

new_data = [[92, 137.0, 6.2, 4.64, 58, 61.0, 256.0, 190.0, 92.0, 1, 1, 0, 0]]

lm.predict(new_data)



array([[ 464.00215102]])